# `propto_omega`

## $\tilde{\alpha}_B$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, alphaB
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.90, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [0.0, 2.5]  # alphaB
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, alphaB):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        :alphaB (float): Braiding parameter in hi_class
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'propto_omega',
        'expansion_model': 'lcdm',
        'expansion_smg': 0, #this value will be overwritten using the closure equation.
        'parameters_smg': f'1., {alphaB}, 0., 0., 1.', # alphas = {K, B, M, T, M*}     
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    cosmo.set(class_params)

    try:
        cosmo.compute()
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            # set YHe to 0.25. Value is from https://arxiv.org/abs/1503.08146 and Plank 2018(Section 7.6.1) https://arxiv.org/abs/1807.06209
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            class_params['YHe'] = 0.25
            cosmo.set(class_params)
            cosmo.compute()
        else:
            raise e

    plin_class = np.array([cosmo.pk_lin(kk*h, 0) for kk in k]) * h ** 3
    
    # Memory cleanup for class
    cosmo.struct_cleanup()
    cosmo.empty()

    return plin_class

In [5]:
%%time
# Compute the linear power spectrum using CLASS
plin_class = [get_class_linear(ks, *t) for t in thetas]

CPU times: user 3min 13s, sys: 8.27 s, total: 3min 21s
Wall time: 59.6 s


In [6]:
np.savetxt("./Data/cosmo_params_propto_omega_B.txt", thetas, header="h omega_b omega_m ns 10^9As alphaB", comments='', fmt="%.8f")

In [7]:
output_dir = "./Data/spectra_latin_hc_propto_omega_B/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(plin_class):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

## $\tilde{\alpha}_M$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, alphaM
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.90, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [0.0, 4]  # alphaM
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, alphaM):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        :alphaM (float): Mass parameter in hi_class
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'propto_omega',
        'expansion_model': 'lcdm',
        'expansion_smg': 0, #this value will be overwritten using the closure equation.
        'parameters_smg': f'1., 0., {alphaM}, 0., 1.', # alphas = {K, B, M, T, M*}     
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    cosmo.set(class_params)

    try:
        cosmo.compute()
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            # set YHe to 0.25. Value is from https://arxiv.org/abs/1503.08146 and Plank 2018(Section 7.6.1) https://arxiv.org/abs/1807.06209
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            class_params['YHe'] = 0.25
            cosmo.set(class_params)
            cosmo.compute()
        else:
            raise e

    plin_class = np.array([cosmo.pk_lin(kk*h, 0) for kk in k]) * h ** 3
    
    # Memory cleanup for class
    cosmo.struct_cleanup()
    cosmo.empty()

    return plin_class

In [5]:
%%time
# Compute the linear power spectrum using CLASS
plin_class = [get_class_linear(ks, *t) for t in thetas]

CPU times: user 4min 28s, sys: 8.01 s, total: 4min 36s
Wall time: 1min 2s


In [6]:
np.savetxt("./Data/cosmo_params_propto_omega_M.txt", thetas, header="h omega_b omega_m ns 10^9As alphaM", comments='', fmt="%.8f")

In [7]:
output_dir = "./Data/spectra_latin_hc_propto_omega_M/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(plin_class):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

## $\tilde{\alpha}_T$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, alphaT
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.90, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [-1.0, 0.0]  # alphaT
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, alphaT):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        :alphaM (float): Mass parameter in hi_class
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'propto_omega',
        'expansion_model': 'lcdm',
        'expansion_smg': 0, #this value will be overwritten using the closure equation.
        'parameters_smg': f'1., 0., 0., {alphaT}, 1.', # alphas = {K, B, M, T, M*}     
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    cosmo.set(class_params)

    try:
        cosmo.compute()
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            # set YHe to 0.25. Value is from https://arxiv.org/abs/1503.08146 and Plank 2018(Section 7.6.1) https://arxiv.org/abs/1807.06209
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            class_params['YHe'] = 0.25
            cosmo.set(class_params)
            cosmo.compute()
        else:
            raise e

    plin_class = np.array([cosmo.pk_lin(kk*h, 0) for kk in k]) * h ** 3
    
    # Memory cleanup for class
    cosmo.struct_cleanup()
    cosmo.empty()

    return plin_class

In [5]:
%%time
# Compute the linear power spectrum using CLASS
plin_class = [get_class_linear(ks, *t) for t in thetas]

CPU times: user 2min 28s, sys: 9.05 s, total: 2min 37s
Wall time: 57.4 s


In [6]:
np.savetxt("./Data/cosmo_params_propto_omega_T.txt", thetas, header="h omega_b omega_m ns 10^9As alphaT", comments='', fmt="%.8f")

In [7]:
output_dir = "./Data/spectra_latin_hc_propto_omega_T/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(plin_class):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

# `propto_scale`

## $\hat{\alpha}_B$

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [9]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, alphaB
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.90, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [0.0, 0.6]  # alphaB
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [10]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [11]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, alphaB):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        :alphaB (float): Braiding parameter in hi_class
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'propto_scale',
        'expansion_model': 'lcdm',
        'expansion_smg': 0., #this value will be overwritten using the closure equation.
        'parameters_smg': f'1., {alphaB}, 0., 0., 1.', # alphas = {K, B, M, T, M*}     
        'skip_stability_tests_smg': 'yes',
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    cosmo.set(class_params)

    try:
        cosmo.compute()
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            # set YHe to 0.25. Value is from https://arxiv.org/abs/1503.08146 and Plank 2018(Section 7.6.1) https://arxiv.org/abs/1807.06209
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            class_params['YHe'] = 0.25
            cosmo.set(class_params)
            cosmo.compute()
        else:
            raise e

    plin_class = np.array([cosmo.pk_lin(kk*h, 0) for kk in k]) * h ** 3
    
    # Memory cleanup for class
    cosmo.struct_cleanup()
    cosmo.empty()

    return plin_class

In [12]:
%%time
# Compute the linear power spectrum using CLASS
plin_class = [get_class_linear(ks, *t) for t in thetas]

CPU times: user 2min 30s, sys: 11.1 s, total: 2min 41s
Wall time: 58.3 s


In [13]:
np.savetxt("./Data/cosmo_params_propto_scale_B.txt", thetas, header="h omega_b omega_m ns 10^9As alphaB", comments='', fmt="%.8f")

In [14]:
output_dir = "./Data/spectra_latin_hc_propto_scale_B/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(plin_class):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

## $\hat{\alpha}_M$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, alphaM
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.90, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [0.0, 1.0]  # alphaM
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, alphaM):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        :alphaM (float): Mass parameter in hi_class
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'propto_omega',
        'expansion_model': 'lcdm',
        'expansion_smg': 0, #this value will be overwritten using the closure equation.
        'parameters_smg': f'1., 0., {alphaM}, 0., 1.', # alphas = {K, B, M, T, M*}     
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    cosmo.set(class_params)

    try:
        cosmo.compute()
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            # set YHe to 0.25. Value is from https://arxiv.org/abs/1503.08146 and Plank 2018(Section 7.6.1) https://arxiv.org/abs/1807.06209
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            class_params['YHe'] = 0.25
            cosmo.set(class_params)
            cosmo.compute()
        else:
            raise e

    plin_class = np.array([cosmo.pk_lin(kk*h, 0) for kk in k]) * h ** 3
    
    # Memory cleanup for class
    cosmo.struct_cleanup()
    cosmo.empty()

    return plin_class

In [5]:
%%time
# Compute the linear power spectrum using CLASS
plin_class = [get_class_linear(ks, *t) for t in thetas]

CPU times: user 2min 58s, sys: 8.55 s, total: 3min 6s
Wall time: 58.2 s


In [6]:
np.savetxt("./Data/cosmo_params_propto_scale_M.txt", thetas, header="h omega_b omega_m ns 10^9As alphaM", comments='', fmt="%.8f")

In [7]:
output_dir = "./Data/spectra_latin_hc_propto_scale_M/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(plin_class):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

## $\hat{\alpha}_T$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, alphaT
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.90, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [-1.0, 0.0]  # alphaT
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, alphaT):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        :alphaM (float): Mass parameter in hi_class
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'propto_scale',
        'expansion_model': 'lcdm',
        'expansion_smg': 0, #this value will be overwritten using the closure equation.
        'parameters_smg': f'1., 0., 0., {alphaT}, 1.', # alphas = {K, B, M, T, M*}     
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    cosmo.set(class_params)

    try:
        cosmo.compute()
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            # set YHe to 0.25. Value is from https://arxiv.org/abs/1503.08146 and Plank 2018(Section 7.6.1) https://arxiv.org/abs/1807.06209
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            class_params['YHe'] = 0.25
            cosmo.set(class_params)
            cosmo.compute()
        else:
            raise e

    plin_class = np.array([cosmo.pk_lin(kk*h, 0) for kk in k]) * h ** 3
    
    # Memory cleanup for class
    cosmo.struct_cleanup()
    cosmo.empty()

    return plin_class

In [5]:
%%time
# Compute the linear power spectrum using CLASS
plin_class = [get_class_linear(ks, *t) for t in thetas]

CPU times: user 2min 52s, sys: 10.4 s, total: 3min 3s
Wall time: 59.9 s


In [6]:
np.savetxt("./Data/cosmo_params_propto_scale_T.txt", thetas, header="h omega_b omega_m ns 10^9As alphaT", comments='', fmt="%.8f")

In [7]:
output_dir = "./Data/spectra_latin_hc_propto_scale_T/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(plin_class):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

# $\mathrm{Galileons}$

## $\mathrm{Cubic~Galileons}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.92, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'galileon',
        'gravity_submodel': 'cubic',
        'cs2_safe_smg': 1e-8,
        'a_min_stability_test_smg': 1e-10,
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    try:
        cosmo.set(class_params)
        cosmo.compute()
        plin_class = np.array([cosmo.pk_lin(kk * h, 0) for kk in k]) * h ** 3
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            try:
                class_params['YHe'] = 0.25
                cosmo.set(class_params)
                cosmo.compute()
                plin_class = np.array([cosmo.pk_lin(kk * h, 0) for kk in k]) * h ** 3
            except Exception:
                return None
        else:
            return None
    except Exception:
        return None
    finally:
        cosmo.struct_cleanup()
        cosmo.empty()

    return plin_class

In [5]:
results = []
valid_params = []

for theta in thetas:
    pk = get_class_linear(ks, *theta)
    if pk is not None:
        results.append(pk)
        valid_params.append(theta)

In [6]:
len(valid_params), len(results)

(65, 65)

In [7]:
np.savetxt("./Data/cosmo_params_cubic_galileon.txt", valid_params, header="h omega_b omega_m ns 10^9As", comments='', fmt="%.8f")

In [8]:
output_dir = "./Data/spectra_latin_hc_cubic_galileon/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(results):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

## $\mathrm{Quartic~Galileons}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, xi
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.92, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [2.39, 2.43]   # xi
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, xi):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'galileon',
        'gravity_submodel': 'quartic',
        'parameters_smg': xi,
        'cs2_safe_smg': 1e-8,
        'a_min_stability_test_smg': 1e-10,
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    try:
        cosmo.set(class_params)
        cosmo.compute()
        plin_class = np.array([cosmo.pk_lin(kk * h, 0) for kk in k]) * h ** 3
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            try:
                class_params['YHe'] = 0.25
                cosmo.set(class_params)
                cosmo.compute()
                plin_class = np.array([cosmo.pk_lin(kk * h, 0) for kk in k]) * h ** 3
            except Exception:
                return None
        else:
            return None
    except Exception:
        return None
    finally:
        cosmo.struct_cleanup()
        cosmo.empty()

    return plin_class

In [6]:
results = []
valid_params = []

for theta in thetas:
    pk = get_class_linear(ks, *theta)
    if pk is not None:
        results.append(pk)
        valid_params.append(theta)

In [7]:
len(valid_params), len(results)

(50, 50)

In [8]:
np.savetxt("./Data/cosmo_params_quartic_galileon.txt", valid_params, header="h omega_b omega_m ns 10^9As xi", comments='', fmt="%.8f")

In [9]:
output_dir = "./Data/spectra_latin_hc_quartic_galileon/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(results):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

## $\mathrm{Quintic~Galileons}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, xi, c3
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.92, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [2.39, 2.43],   # xi
                 [-0.132, -0.2]   # c3
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, xi, c3):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_Lambda': 0,
        'Omega_fld': 0,
        'Omega_smg': -1,
        'gravity_model': 'galileon',
        'gravity_submodel': 'quintic',
        'parameters_smg': f'{xi}, {c3}',
        'cs2_safe_smg': 1e-8,
        'a_min_stability_test_smg': 1e-10,
        'skip_stability_test_smg': 'yes',
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    try:
        cosmo.set(class_params)
        cosmo.compute()
        plin_class = np.array([cosmo.pk_lin(kk * h, 0) for kk in k]) * h ** 3
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            try:
                class_params['YHe'] = 0.25
                cosmo.set(class_params)
                cosmo.compute()
                plin_class = np.array([cosmo.pk_lin(kk * h, 0) for kk in k]) * h ** 3
            except Exception:
                return None
        else:
            return None
    except Exception:
        return None
    finally:
        cosmo.struct_cleanup()
        cosmo.empty()

    return plin_class

In [5]:
results = []
valid_params = []

for theta in thetas:
    pk = get_class_linear(ks, *theta)
    if pk is not None:
        results.append(pk)
        valid_params.append(theta)

In [6]:
len(valid_params), len(results)

(0, 0)

In [7]:
np.savetxt("./Data/cosmo_params_quartic_galileon.txt", valid_params, header="h omega_b omega_m ns 10^9As xi", comments='', fmt="%.8f")

In [8]:
output_dir = "./Data/spectra_latin_hc_quartic_galileon/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(results):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

## $f(R)$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, log10fR0
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.90, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [-7.0, -4.0]  # log10fR0
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, log10fR0):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        :log10fR0 (float): Parameter of Hu-Sawicki f(R) model
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_fld': 0,
        'Omega_scf': 0,
        'mg_ansatz': 'FR',
        'F_R0': 10**log10fR0, 
        'FRn': 1,
        'gauge':'newtonian',
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    cosmo.set(class_params)

    try:
        cosmo.compute()
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            # set YHe to 0.25. Value is from https://arxiv.org/abs/1503.08146 and Plank 2018(Section 7.6.1) https://arxiv.org/abs/1807.06209
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            class_params['YHe'] = 0.25
            cosmo.set(class_params)
            cosmo.compute()
        else:
            raise e

    plin_class = np.array([cosmo.pk_lin(kk*h, 0) for kk in k]) * h ** 3
    
    # Memory cleanup for class
    cosmo.struct_cleanup()
    cosmo.empty()

    return plin_class

In [5]:
%%time
# Compute the linear power spectrum using CLASS
plin_class = [get_class_linear(ks, *t) for t in thetas]

    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the FR model
    ******************************************
    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the FR model
    ******************************************
    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the FR model
    ******************************************
    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the FR model
    ******************************************
    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the FR mod

In [6]:
np.savetxt("./Data/cosmo_params_fR.txt", thetas, header="h omega_b omega_m ns 10^9As log10fR0", comments='', fmt="%.8f")

In [7]:
output_dir = "./Data/spectra_latin_hc_fR/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(plin_class):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')

## `plk_late`

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import os

In [2]:
def get_param_samples(n, seed):
    """
    Get n samples from a latin hypercube from uniform prior
    
    Args:
        :n (int): Number of sets of cosmological samples to obtain
        :seed (int): The random seems for the latin hypercube
        
    Returns:
        :cosmo_params (np.ndarray): Array of cosmological parameters of shape (n, number of parameters).
            The parameters are ordered h, omega_b, omega_m, ns, 10^9 As, E11, E22
    """
    
    all_prior = [[0.65, 0.75],    # h
                 [0.0214, 0.0234],  # omega_b 
                 [0.13, 0.15],  # omega_m 
                 [0.90, 1.00],  # ns
                 [1.5, 2.5],  # As 10⁹
                 [-1.0, 1.0],  # E11
                 [-1.0, 1.0]   # E22
                ]

    # Generate samples
    d = len(all_prior)
    lh_sampler = stats.qmc.LatinHypercube(d, seed=seed)
    cosmo_params = lh_sampler.random(n)
    for i in range(d):
        cosmo_params[:,i] = all_prior[i][0] + (all_prior[i][1] - all_prior[i][0]) * cosmo_params[:,i]
        
    return cosmo_params

In [3]:
#generate cosmological parameter samples
thetas = get_param_samples(100, 1)

#set up k values
kmin = 1e-4
kmax = 1.5
nk = 200
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

In [4]:
# set up the CLASS simulator for comparison
from classy import Class
from classy import CosmoComputationError
import warnings

def get_class_linear(k, h, omega_b, omega_m, ns, As_e9, E11, E22):
    """
    Compute linear P(k) using class for the cosmology of interest
    
    Args:
        :k (np.ndarray): k values in [h / Mpc]
        :omega_b (float): baryonic reduced density parameter,
        :omega_m (float): The z=0 total matter reduced density parameter,
        :h (float): reduced Hubble constant
        :ns (float): Spectral tilt of primordial power spectrum
        :As_e9 (float): 10^9 times the amplitude of the primordial P(k)
        :E11 (float): Parameters of plk_late model
        :E22 (float): Parameters of plk_late model
        
    Returns:
        :plin_class (np.ndarray): Linear power spectrum at corresponding k values [(Mpc/h)^3]
    """
    
    class_params = {
        'h': h,
        'omega_b': omega_b,
        'omega_cdm': omega_m - omega_b,
        'n_s': ns,
        'A_s': As_e9*1.e-9,
        'Omega_fld': 0,
        'Omega_scf': 0,
        'mg_ansatz': 'plk_late',
        'mg_E11': E11, 
        'mg_E22': E22,
        'gauge':'newtonian',
        'output': 'mPk',
        'P_k_max_1/Mpc': k.max() * h,
    }
        
    cosmo = Class()
    cosmo.set(class_params)

    try:
        cosmo.compute()
    except CosmoComputationError as e:
        if "DeltaNeff < deltaN[0]" in str(e):
            # set YHe to 0.25. Value is from https://arxiv.org/abs/1503.08146 and Plank 2018(Section 7.6.1) https://arxiv.org/abs/1807.06209
            warnings.warn(f"Adjust YHe to 0.25 due to CLASS CosmoComputationError for cosmology {class_params}.")
            class_params['YHe'] = 0.25
            cosmo.set(class_params)
            cosmo.compute()
        else:
            raise e

    plin_class = np.array([cosmo.pk_lin(kk*h, 0) for kk in k]) * h ** 3
    
    # Memory cleanup for class
    cosmo.struct_cleanup()
    cosmo.empty()

    return plin_class

In [5]:
%%time
# Compute the linear power spectrum using CLASS
plin_class = [get_class_linear(ks, *t) for t in thetas]

    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the plk_late model
    ******************************************
    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the plk_late model
    ******************************************
    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the plk_late model
    ******************************************
    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000    
       For the plk_late model
    ******************************************
    ******************************************
       Modified Gravity extensions enabled:  
       Modifications active for z < 6.000  

In [6]:
np.savetxt("./Data/cosmo_params_plk_late.txt", thetas, header="h omega_b omega_m ns 10^9As E11 E22", comments='', fmt="%.8f")

In [7]:
output_dir = "./Data/spectra_latin_hc_plk_late/"
os.makedirs(output_dir, exist_ok=True)

# Save each spectrum as a txt file with two columns: k and P(k)
for i, pk in enumerate(plin_class):
    fname = os.path.join(output_dir, f"spectrum_{1+i:03d}.txt")
    np.savetxt(fname, np.column_stack([ks, pk]), header="k  P(k)", comments='')